In [1]:
stock = 'AMZN'
days_back = 100

In [2]:
import pandas as pd                              # 表格和数据处理
import matplotlib.pyplot as plt                  # 绘图
import seaborn as sns                            # 更多绘图功能
import pandas_datareader.data as webdata

from itertools import product                    # 一些有用的函数
from tqdm import tqdm_notebook

import datetime                                  # 日期函数

import warnings                                  # 勿扰模式
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
now = datetime.date.today()
start = now - datetime.timedelta(days=days_back)
# df = data.DataReader(stock, "iex", start=start, end=end)
df = webdata.get_data_yahoo(stock, start=start, end=now)['Adj Close']

In [4]:
df

Date
2018-07-23    1802.000000
2018-07-24    1829.239990
2018-07-25    1863.609985
2018-07-26    1808.000000
2018-07-27    1817.270020
2018-07-30    1779.219971
2018-07-31    1777.439941
2018-08-01    1797.170044
2018-08-02    1834.329956
2018-08-03    1823.290039
2018-08-06    1847.750000
2018-08-07    1862.479980
2018-08-08    1886.520020
2018-08-09    1898.520020
2018-08-10    1886.300049
2018-08-13    1896.199951
2018-08-14    1919.650024
2018-08-15    1882.619995
2018-08-16    1886.520020
2018-08-17    1882.219971
2018-08-20    1876.709961
2018-08-21    1883.420044
2018-08-22    1904.900024
2018-08-23    1902.900024
2018-08-24    1905.390015
2018-08-27    1927.680054
2018-08-28    1932.819946
2018-08-29    1998.099976
2018-08-30    2002.380005
2018-08-31    2012.709961
                 ...     
2018-09-18    1941.050049
2018-09-19    1926.420044
2018-09-20    1944.300049
2018-09-21    1915.010010
2018-09-24    1934.359985
2018-09-25    1974.550049
2018-09-26    1974.849976
2018-09

In [5]:
# 需要预测的5天日期
d = [None] * 5
j = 0

for i in range(0,7):
    if (now + datetime.timedelta(i)).weekday() < 5:
        d[j] = str(now + datetime.timedelta(i))
        j = j + 1

In [6]:
#时序数据上的线性模型
data = pd.DataFrame(df.copy().astype(float))
data.columns = ["y"]

df = pd.DataFrame({'y':[0,0,0,0,0]},
                   index = d)
data = data.append(df)

for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

y = data.dropna().y # 删除n/a的数据
X = data.dropna().drop(['y'], axis=1)

# 除去需要预测的5天
train_index = int(len(X)) - 5
X_train = X.iloc[:train_index]
y_train = y.iloc[:train_index]

# 机器学习
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X.tail(5))
precentage = prediction.mean()
print(prediction.mean())
print(prediction)
# plt.figure(figsize=(15, 7))
# plt.plot(prediction, "g", label="prediction", linewidth=2.0)


1555.3609965034789
[1617.48291347 1567.24080821 1424.79514487 1594.56181669 1572.72429927]


In [7]:
#储存数据到excel
from openpyxl import load_workbook
df = pd.DataFrame(prediction)
df = df.values.T.tolist()
stock_name = [str(stock)]
wb = load_workbook('/Users/BartSu/Desktop/StockRS_Result.xlsx')
ws = wb.worksheets[0]

ws.append(stock_name)

# Append 2 new Rows - Columns A - D
for row_data in df:
    # Append Row Values
    ws.append(row_data)

wb.save("/Users/BartSu/Desktop/StockRS_Result.xlsx")